Regresion lineal multiple

Importar las librerias necesarias

In [2]:
# utilizado para manejos de directorios y rutas
import os

# Computacion vectorial y cientifica para python
import numpy as np

#Importacion de pandas para cargar el DataSet
import pandas as pd

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # Necesario para graficar superficies 3D

# llama a matplotlib a embeber graficas dentro de los cuadernillos
%matplotlib inline

Importar y cargar el DataSet

In [3]:
#importamos el dataset
FileDs = r'C:\Users\jhean\OneDrive\Documentos\Tareas\Inteligencia artificial\DataSets\workout_fitness_tracker_data.csv'
# Cargamos el dataset
data = pd.read_csv(FileDs, delimiter=',')
data["Workout Intensity"] = data["Workout Intensity"].map({"low": 1, "medium": 2, "high": 3})#Convertimos los datos de Workout Intensity a valores numericos
X = data.drop(columns=["User ID", "Calories Burned", "Gender", "Workout Type", "Mood Before Workout", "Mood After Workout"]).values  #Filtrado de columnas no relevantes para inferir en y.
y = data["Calories Burned"].values  # Variable a inferir en este caso las calorias quemadas
m = y.size  # m = numero de ejemplos de entrenamiento

Verificacion de carga de datos

In [ ]:
print(data)

In [ ]:
print(X)

In [ ]:
print(y)

Generacion de graficas

In [ ]:
#Importamos lo necesario
import matplotlib.pyplot as plt
import seaborn as sns
#Visualizamos la distribución de las calorias quemadas
plt.figure(figsize=(8,6))
sns.histplot(y, bins=30, kde=True, color='skyblue')
plt.title('Distribución de Calorías Quemadas')
plt.xlabel('Calorías Quemadas')
plt.ylabel('Frecuencia')
plt.show()
